# Unity ML Agents
## Proximal Policy Optimization (PPO)
Contains an implementation of PPO as described [here](https://arxiv.org/abs/1707.06347).

In [1]:
import numpy as np
import os
import tensorflow as tf

from ppo.history import *
from ppo.models import *
from ppo.trainer import Trainer
from unityagents import *

### Hyperparameters

In [2]:
### General parameters
max_steps = 35e5 # Set maximum number of steps to run environment.
run_path = "ppo_wind" # The sub-directory name for model and summary statistics
load_model = True # Whether to load a saved model.
train_model = True # Whether to train the model.
summary_freq = 10000 # Frequency at which to save training statistics.
save_freq = 50000 # Frequency at which to save model.
env_name = "pong_wind" # Name of the training environment file.

### Algorithm-specific parameters for tuning
gamma = 0.99 # Reward discount rate.
lambd = 0.95 # Lambda parameter for GAE.
time_horizon = 2048 # How many steps to collect per agent before adding to buffer.
beta = 1e-3 # Strength of entropy regularization
num_epoch = 5 # Number of gradient descent steps per batch of experiences.
epsilon = 0.2 # Acceptable threshold around ratio of old and new policy probabilities.
buffer_size = 2048 # How large the experience buffer should be before gradient descent.
learning_rate = 1e-4 # Model learning rate.
hidden_units = 64 # Number of units in hidden layer.
batch_size = 64 # How many experiences per gradient descent update step.

### Load the environment

In [3]:
env = UnityEnvironment(file_name=env_name)
print(str(env))
brain_name = env.brain_names[0]

INFO:unityagents.environment:
'PongAcademy' started successfully!


Unity Academy name: PongAcademy
        Number of brains: 1
        Reset Parameters :
		
Unity brain name: PongBrain
        Number of observations (per agent): 0
        State space type: continuous
        State space size (per agent): 8
        Action space type: continuous
        Action space size (per agent): 2
        Memory space size (per agent): 0
        Action descriptions: x_rotate, z_rotate


### Train the Agent(s)

In [4]:
tf.reset_default_graph()

# Create the Tensorflow model graph
ppo_model = create_agent_model(env, lr=learning_rate,
                               h_size=hidden_units, epsilon=epsilon,
                               beta=beta, max_step=max_steps)

is_continuous = (env.brains[brain_name].action_space_type == "continuous")
use_observations = (env.brains[brain_name].number_observations > 0)
use_states = (env.brains[brain_name].state_space_size > 0)

model_path = './models/{}'.format(run_path)
summary_path = './summaries/{}'.format(run_path)

if not os.path.exists(model_path):
    os.makedirs(model_path)

if not os.path.exists(summary_path):
    os.makedirs(summary_path)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    # Instantiate model parameters
    if load_model:
        print('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(init)
    steps = sess.run(ppo_model.global_step)
    summary_writer = tf.summary.FileWriter(summary_path)
    info = env.reset(train_mode=train_model)[brain_name]
    trainer = Trainer(ppo_model, sess, info, is_continuous, use_observations, use_states)
    while steps <= max_steps:
        if env.global_done:
            info = env.reset(train_mode=train_model)[brain_name]
        # Decide and take an action
        new_info = trainer.take_action(info, env, brain_name)
        info = new_info
        trainer.process_experiences(info, time_horizon, gamma, lambd)
        if len(trainer.training_buffer['actions']) > buffer_size and train_model:
            # Perform gradient descent with experience buffer
            trainer.update_model(batch_size, num_epoch)
        if steps % summary_freq == 0 and steps != 0 and train_model:
            # Write training statistics to tensorboard.
            trainer.write_summary(summary_writer, steps)
        if steps % save_freq == 0 and steps != 0 and train_model:
            # Save Tensorflow model
            save_model(sess, model_path=model_path, steps=steps, saver=saver)
        steps += 1
        sess.run(ppo_model.increment_step)
    # Final save Tensorflow model
    if steps != 0 and train_model:
        save_model(sess, model_path=model_path, steps=steps, saver=saver)
env.close()
export_graph(model_path, env_name)

Loading Model...
INFO:tensorflow:Restoring parameters from ./models/ppo_wind\model-2450000.cptk


INFO:tensorflow:Restoring parameters from ./models/ppo_wind\model-2450000.cptk
c:\users\niles\anaconda2\envs\ml-agents\lib\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\users\niles\anaconda2\envs\ml-agents\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Mean Reward: nan
Saved Model
Mean Reward: 10.879879709385925
Mean Reward: 9.46524049241424
Mean Reward: 15.950701609192855
Mean Reward: 14.146819595787058
Mean Reward: 13.467401845317474
Saved Model
Mean Reward: 11.508646859307209
Mean Reward: 16.266712965354742
Mean Reward: 12.737212650354367
Mean Reward: 12.403202352372935
Mean Reward: 21.09162005081437
Saved Model
Mean Reward: 8.746325073187212
Mean Reward: 11.772374663908053
Mean Reward: 18.31317798857413
Mean Reward: 20.469890155435362
Mean Reward: 10.668777768771506
Saved Model
Mean Reward: 13.100830623396977
Mean Reward: 10.024443534829587
Mean Reward: 23.446146117406943
Mean Reward: 21.980997945658313
Mean Reward: 26.93546790955476
Saved Model
Mean Reward: 25.11787896583384
Mean Reward: 18.781897533690024
Mean Reward: 4.822904101619503
Mean Reward: 18.370016897329474
Mean Reward: 16.636392854379647
Saved Model
Mean Reward: 12.867708515522649
Mean Reward: 27.56214940577874
Mean Reward: 19.269856917360396
Mean Reward: 21.19991710

INFO:tensorflow:Restoring parameters from ./models/ppo_wind\model-3500001.cptk


INFO:tensorflow:Froze 4 variables.


INFO:tensorflow:Froze 4 variables.


Converted 4 variables to const ops.


### Export the trained Tensorflow graph
Once the model has been trained and saved, we can export it as a .bytes file which Unity can embed.

In [5]:
export_graph(model_path, env_name)

INFO:tensorflow:Restoring parameters from ./models/ppo\model-400000.cptk


INFO:tensorflow:Restoring parameters from ./models/ppo\model-400000.cptk


INFO:tensorflow:Froze 4 variables.


INFO:tensorflow:Froze 4 variables.


Converted 4 variables to const ops.
